<a href="https://colab.research.google.com/github/EltonWolf/GoogleEarthPythonBootcampAPI-/blob/main/Applying_Landsat_Simple_Composite_Algorithm_to_clipped_Area.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=0MnS_t4SN2YzMuaygVK0ZxilHkFgdOdy6CvRe0nxIZA&tc=Q9e4ZtsWgRpIpQky_QMz-lf145yBvH-yHn6ixm-Evcg&cc=1bww3-SXmRRPXQb-lalk5LUlR924eM-1WYr6PqpCWDU

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXl-gwGZIdMt-mbU9Z4Dkh3p-oJBhOLezjpTb6mze8b6S9Q5tbZS3H4

Successfully saved authorization token.


In [2]:
# Calculating 6 months of Landsat - Composite
landsat8 = ee.ImageCollection("LANDSAT/LC08/C02/T1")

CA = ee.FeatureCollection('TIGER/2018/States').filter(ee.Filter.eq('STUSPS', 'CA'))

In [5]:
#Apply simple composite algorithm
composite = ee.Algorithms.Landsat.simpleComposite(**{
    'collection' : landsat8.filterDate('2017-01-01', '2017-12-31')
    'asFloat' : True
})

In [10]:
# Load the US states feature collection
us_states = ee.FeatureCollection('TIGER/2018/States')

# Filter to get only California
CA = us_states.filter(ee.Filter.eq('STUSPS', 'CA'))

# Clip the composite image to the boundaries of California
composite_clipped = composite.clipToCollection(CA)

In [17]:
# Import the Folium library.
import folium

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Create a folium map object.
my_map = folium.Map(location=[40, -120.649], zoom_start = 5)

# Add the layer to the map object.
my_map.add_ee_layer(composite_clipped, {'bands': ['B6', 'B5', 'B4'], 'max': [0.3, 0.4, 0.3]}, "Landsat 8 6 Month Composite")

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)